In [4]:
import pickle
import os
import pickle
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
from google.oauth2 import service_account

%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [5]:
# This cell is just about making things look nice
# Make plots bigger by default.
# Has to occur after the %matplotlib inline in a different cell. Known issue
plt.rc('figure', figsize=(14.0, 8.0))
matplotlib.rcParams.update({'font.size': 22})

# Make numpy outputs easier to read
np.set_printoptions(precision=3, formatter={'float': '{: 8.3f}'.format})

In [6]:
## Set up bigquery environment

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ubuntu/certs/stackoverflow-metis-32ecb3fa04c0.json"
project_id = 'stackoverflow-metis'
client = bigquery.Client(project=project_id)
dataset_id = 'SO_dataset'
dataset_ref = client.dataset(dataset_id)
dataset = client.get_dataset(dataset_ref)

In [ ]:
## Create a view to reconstruct user reputation at 01/01/2017 and 01/01/2018
view_ref = dataset_ref.table('SO_user_reputation_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
SELECT
  user.id,
  LENGTH(about_me) AS about_me_length,
  user.age,
  user.creation_date,
  user.last_access_date,
  user.location,
  user.reputation,
  user.up_votes,
  user.down_votes,
  user.profile_image_url,
  user.website_url,
  answer_2017.answer_reputation_total_2017,
  question_2017.question_reputation_total_2017,
  accepted_2017.accepted_reputation_total_2017,
  answer_2018.answer_reputation_total_2018,
  question_2018.question_reputation_total_2018,
  accepted_2018.accepted_reputation_total_2018
FROM
  `bigquery-public-data.stackoverflow.users` AS user
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(answer_reputation) AS answer_reputation_total_2017
  FROM (
    SELECT
      owner_user_id,
      score * 10 AS answer_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    WHERE
      creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS answer_2017
ON
  user.id = answer_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(question_reputation) AS question_reputation_total_2017
  FROM (
    SELECT
      owner_user_id,
      score * 5 AS question_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    WHERE
      creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS question_2017
ON
  user.id = question_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(accepted_reputation) AS accepted_reputation_total_2017
  FROM (
    SELECT
      answer.owner_user_id,
      CASE
        WHEN answer.id = question.accepted_answer_id THEN 15
        ELSE 0
      END AS accepted_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers` AS answer
    LEFT JOIN
      `bigquery-public-data.stackoverflow.posts_questions` question
    ON
      answer.parent_id = question.id
    WHERE
      answer.creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS accepted_2017
ON
  user.id = accepted_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(answer_reputation) AS answer_reputation_total_2018
  FROM (
    SELECT
      owner_user_id,
      score * 10 AS answer_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    WHERE
      creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS answer_2018
ON
  user.id = answer_2018.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(question_reputation) AS question_reputation_total_2018
  FROM (
    SELECT
      owner_user_id,
      score * 5 AS question_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    WHERE
      creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS question_2018
ON
  user.id = question_2018.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(accepted_reputation) AS accepted_reputation_total_2018
  FROM (
    SELECT
      answer.owner_user_id,
      CASE
        WHEN answer.id = question.accepted_answer_id THEN 15
        ELSE 0
      END AS accepted_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers` AS answer
    LEFT JOIN
      `bigquery-public-data.stackoverflow.posts_questions` question
    ON
      answer.parent_id = question.id
    WHERE
      answer.creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS accepted_2018
ON
  user.id = accepted_2018.owner_user_id
"""
view = client.create_table(view)

In [13]:
## Retrieve data from the view
query_job = client.query("""
    SELECT *
    FROM `stackoverflow-metis.SO_dataset.SO_user_reputation_view`
    """)

results = query_job.result()
user_reputation = results.to_dataframe()

In [23]:
## Process and save the reputation data
user_reputation['reputation_2017'] = user_reputation[['answer_reputation_total_2017','question_reputation_total_2017','accepted_reputation_total_2017']].sum(axis=1)
user_reputation['reputation_2018'] = user_reputation[['answer_reputation_total_2018','question_reputation_total_2018','accepted_reputation_total_2018']].sum(axis=1)
with open('./data/processed/user_reputation.pkl', 'wb') as picklefile:
    pickle.dump(user_reputation, picklefile)

In [4]:
with open('./data/processed/user_reputation.pkl', 'rb') as picklefile:
    user_reputation = pickle.load(picklefile)

In [5]:
user_reputation.tail(100)

,id,about_me_length,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,profile_image_url,website_url,answer_reputation_total_2017,question_reputation_total_2017,accepted_reputation_total_2017,answer_reputation_total_2018,question_reputation_total_2018,accepted_reputation_total_2018,reputation_2017,reputation_2018
8917407,1525495,98,,2012-07-14 11:28:27.330000+00:00,2018-06-03 04:39:12.047000+00:00,Spain,6172,884,201,https://i.stack.imgur.com/DunHU.png,https://llorxscript.blogspot.com.es,2850.0,230.0,750.0,3900.0,255.0,1395.0,3830.0,5550.0
8917408,1811348,0,,2012-11-09 05:53:59.980000+00:00,2018-06-01 15:31:25.783000+00:00,,797,582,1,,,320.0,225.0,150.0,320.0,255.0,150.0,695.0,725.0
8917409,379532,0,,2010-06-29 22:54:24.830000+00:00,2018-06-02 21:04:55.380000+00:00,,430,81,0,,,70.0,255.0,30.0,70.0,255.0,30.0,355.0,355.0
8917410,2985898,0,,2013-11-13 03:01:55.323000+00:00,2018-05-30 05:43:25.747000+00:00,,437,15,5,https://www.gravatar.com/avatar/?s=128&d=identicon&r=PG&f=1,,70.0,235.0,15.0,70.0,255.0,15.0,320.0,340.0
8917411,1816631,0,,2012-11-11 20:21:58.540000+00:00,2018-05-24 11:46:48.353000+00:00,,521,50,0,,,70.0,245.0,60.0,70.0,255.0,60.0,375.0,385.0
8917412,801169,0,,2011-06-16 09:34:42.970000+00:00,2013-02-24 17:46:32.397000+00:00,,340,84,0,,,70.0,255.0,0.0,70.0,255.0,0.0,325.0,325.0
8917413,22227,0,,2008-09-25 15:40:21.703000+00:00,2018-05-25 13:01:19.037000+00:00,"San Francisco, CA",7101,214,22,,http://probst.io,6460.0,245.0,120.0,6470.0,255.0,120.0,6825.0,6845.0
8917414,105668,0,,2009-05-12 20:05:21.497000+00:00,2018-06-01 17:24:45.647000+00:00,"Moscow, Russia",1526,39,2,,,1100.0,255.0,30.0,1100.0,255.0,30.0,1385.0,1385.0
8917415,1062015,0,,2011-11-23 13:41:45.487000+00:00,2018-05-30 11:11:20.233000+00:00,"Johannesburg, Gauteng, South Africa",3276,281,24,https://i.stack.imgur.com/BxVmw.jpg,,2380.0,255.0,495.0,2380.0,255.0,495.0,3130.0,3130.0
8917416,187523,0,,2009-10-10 02:37:21.080000+00:00,2018-06-02 23:33:16.100000+00:00,,2257,730,72,https://i.stack.imgur.com/4KSEa.png?s=128&g=1,,1860.0,255.0,90.0,1870.0,255.0,90.0,2205.0,2215.0


In [6]:
query = """
#standardSQL
SELECT
  id,
  COUNT(diff) AS count_diff,
  MAX(diff)/24 AS MAX_activity_interval
FROM (
  SELECT
    id,
    TIMESTAMP_DIFF(creation_date, prev_creation_date, HOUR) AS diff
  FROM (
    SELECT
      id,
      creation_date,
      LAG(creation_date) OVER(PARTITION BY id ORDER BY creation_date ) AS prev_creation_date
    FROM (
      SELECT
        *
      FROM (
        SELECT
          owner_user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.posts_answers`
        UNION ALL
        SELECT
          owner_user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.posts_questions`
        UNION ALL
        SELECT
          user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.comments`)
      WHERE
        FORMAT_TIMESTAMP('%Y', creation_date) = '2017')))
GROUP BY
  id
ORDER BY
  COUNT(diff) DESC
"""
query_job = client.query(query)
results = query_job.result()
inter_activity_days = results.to_dataframe()

In [8]:
inter_activity_days.dropna(axis=0, inplace=True)
inter_activity_days.id = inter_activity_days.id.astype(int)
with open('./data/processed/inter_activity_days.pkl', 'wb') as picklefile:
    pickle.dump(inter_activity_days, picklefile)

In [7]:
query = """
#standardSQL
SELECT
id,
count(id) AS count
FROM (
  SELECT
    *
  FROM (
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017')
GROUP BY
id
"""
query_job = client.query(query)
results = query_job.result()
activity_count = results.to_dataframe()

In [77]:
with open('./data/processed/activity_count.pkl', 'wb') as picklefile:
    pickle.dump(activity_count, picklefile)

In [7]:
query = """
#standardSQL
SELECT
id,
count(id) AS count
FROM (
  SELECT
    *
  FROM (
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017')
GROUP BY
id
"""
query_job = client.query(query)
results = query_job.result()
answer_count = results.to_dataframe()

In [8]:
with open('./data/processed/answer_count.pkl', 'wb') as picklefile:
    pickle.dump(answer_count, picklefile)

In [9]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
  SELECT id, FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM(
    SELECT 
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
) SELECT id,
    COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
    COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
    COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
    COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
    COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
    COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
    COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
    COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
    COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
    COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
    COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
    COUNT(IF(creation_time = "201712", id, NULL)) as m_201712
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
answer_time_series = results.to_dataframe()

In [10]:
answer_time_series.dropna(inplace=True)
answer_time_series['id'] = answer_time_series['id'].astype(int)
with open('./data/processed/answer_time_series.pkl', 'wb') as picklefile:
    pickle.dump(answer_time_series, picklefile)

In [12]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
  SELECT id, FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM(
    SELECT 
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions` UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
) SELECT id,
    COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
    COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
    COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
    COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
    COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
    COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
    COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
    COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
    COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
    COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
    COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
    COUNT(IF(creation_time = "201712", id, NULL)) as m_201712
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
activity_time_series = results.to_dataframe()

In [13]:
activity_time_series.dropna(inplace=True)
activity_time_series['id'] = activity_time_series['id'].astype(int)
with open('./data/processed/activity_time_series.pkl', 'wb') as picklefile:
    pickle.dump(activity_time_series, picklefile)

In [11]:
activity_time_series.shape

(1183423, 13)

In [5]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
  SELECT id, FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM(
    SELECT 
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions` UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2018'
) SELECT id,
    COUNT(DISTINCT IF(creation_time = "201803", id, NULL)) as m_201803,
    COUNT(DISTINCT IF(creation_time = "201804", id, NULL)) as m_201804,
    COUNT(DISTINCT IF(creation_time = "201805", id, NULL)) as m_201805
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
activity_time_series_2018 = results.to_dataframe()

In [11]:
activity_time_series_2018.dropna(inplace=True)
activity_time_series_2018['id'] = activity_time_series_2018['id'].astype(int)
with open('./data/processed/activity_time_series_2018.pkl', 'wb') as picklefile:
    pickle.dump(activity_time_series_2018, picklefile)